In [120]:
# only permit vertex cover

canonical_order = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz',  'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB']
def reorder(df, canonical_order=canonical_order, by='dataset', extras=['dataset','gen_n', 'gen_n_max'], secondary='gen_n', columns=None):
    df['dataset_name_order'] = df[by].map({name: i for i, name in enumerate(canonical_order)})
    if secondary is not None:
        df = df.sort_values(by=['dataset_name_order', secondary])
    else:
        df = df.sort_values(by=['dataset_name_order'])

    df.drop('dataset_name_order', axis =1, inplace=True)

    if columns:
        return df[columns ]
    else:
        return df

In [121]:
task = 'max_cut'
#task = 'vertex_cover'

In [122]:
import os
os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs')

['legacy',
 'Testing',
 '230928_snapshot',
 'LiftMP_runs',
 '230924_hparam2',
 '230927_snapshot',
 '230928_runs',
 '230924_hparam',
 '230924_hparam_TU_multiarch',
 '230924_hparam_TU',
 '230926_finetune_ER_runs']

In [123]:
print(list(os.path.join('230927_snapshot', x) for x in os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/230927_snapshot')))

['230927_snapshot/230925_TUsmall_GAT_VC', '230927_snapshot/230925_TUsmall_GIN_VC', '230927_snapshot/230925_generated_preset_cut', '230927_snapshot/230925_TUlarge_all_cut', '230927_snapshot/230925_TUsmall_GCNN_VC', '230927_snapshot/230925_generated_liftMP_VC', '230927_snapshot/230925_TUlarge_all_VC', '230927_snapshot/230925_generated_preset_VC', '230927_snapshot/230925_TUsmall_liftMP_VC', '230927_snapshot/230925_TUsmall_GatedGCNN_VC', '230927_snapshot/230925_generated_liftMP_cut']


In [124]:
run_folders = ['LiftMP_runs', '230927_snapshot/230925_TUsmall_GAT_VC', '230927_snapshot/230925_TUsmall_GIN_VC', 
               '230927_snapshot/230925_generated_preset_cut', '230927_snapshot/230925_TUlarge_all_cut', '230927_snapshot/230925_TUsmall_GCNN_VC',
                 '230927_snapshot/230925_generated_liftMP_VC', '230927_snapshot/230925_TUlarge_all_VC', '230927_snapshot/230925_generated_preset_VC', 
                 '230927_snapshot/230925_TUsmall_liftMP_VC', '230927_snapshot/230925_TUsmall_GatedGCNN_VC', '230927_snapshot/230925_generated_liftMP_cut']

run_folders = ['230928_snapshot/230925_TUsmall_GAT_VC', '230928_snapshot/230925_TUsmall_GIN_cut', 
               '230928_snapshot/230925_generated_preset_cut', '230928_snapshot/230925_TUsmall_GAT_cut', 
               '230928_snapshot/230925_TUsmall_liftMP_cut', '230928_snapshot/230925_TUsmall_GCNN_VC', 
               '230928_snapshot/230925_TUsmall_GCNN_cut', '230928_snapshot/230925_generated_liftMP_VC', 
               '230928_snapshot/230925_generated_preset_VC', '230928_snapshot/230925_TUsmall_liftMP_VC', '230928_snapshot/230925_TUsmall_GatedGCNN_VC', 
               '230928_snapshot/230925_TUsmall_VC_32', '230928_snapshot/230925_generated_liftMP_cut', '230928_snapshot/230925_TUsmall_GatedGCNN_cut']

run_folders = ['230928_runs/230925_TUsmall_GAT_VC', '230928_runs/230925_TUsmall_GIN_cut', 
               '230928_runs/230925_generated_preset_cut', '230928_runs/230925_TUsmall_GAT_cut', 
               '230928_runs/230925_TUsmall_liftMP_cut', '230928_runs/230925_TUsmall_GCNN_VC', 
               '230928_runs/230925_TUsmall_GCNN_cut', '230928_runs/230925_generated_liftMP_VC', '230928_runs/230925_generated_preset_VC', 
               '230928_runs/230925_TUsmall_liftMP_VC', '230928_runs/230925_TUsmall_GatedGCNN_VC', '230928_runs/230925_TUsmall_VC_32', 
               '230928_runs/230925_TUlarge_liftMP_cut', '230928_runs/230925_TUlarge_liftMP_VC', '230928_runs/230925_generated_liftMP_cut', '230928_runs/230925_TUsmall_GatedGCNN_cut']


In [125]:
import json
import pandas as pd
import numpy as np

folder_path = '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs'
model_list = [os.path.join(folder_path, run_folder, x) for run_folder in run_folders for x in os.listdir(os.path.join(folder_path, run_folder))  ]


rows = []

errored = []
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        if model_args['problem_type'] != task:
            continue
        
        losses = np.load(os.path.join(model_folder, 'valid_scores.npy'))
        test_losses = np.load(os.path.join(model_folder, 'test_scores.npy'))

        modeldict = model_args #{x: model_args[x] for x in params}
        modeldict['max_valid_score'] = max(losses)
        modeldict['max_valid_epoch'] = np.argmax(losses)
        modeldict['scores'] = test_losses[np.argmax(losses)]
        modeldict['baseline'] = False


        # set the time per pred
        time_per_pred = None
        scorefile = [x for x in os.listdir(model_folder) if x.startswith('retest_best')]
        if len(scorefile) >= 1:
            times, _ = np.load(os.path.join(model_folder, scorefile[-1]))
            time_per_pred = np.average(times)
        modeldict['time_per_pred'] = time_per_pred


        # reset test score if the validation is better.
        valid_score_from_file = -np.inf
        test_score_from_file = -np.inf

        for prefix in ['revalidate_best', 'revalidate_last']:
            scorefile = [x for x in os.listdir(model_folder) if x.startswith(prefix)]
            #assert(len(scorefile) <=1)
            if len(scorefile) >= 1:
                _, scores = np.load(os.path.join(model_folder, scorefile[-1]))
                valid_score = np.average(scores)

                if valid_score > valid_score_from_file:
                    testscorefile = [x for x in os.listdir(model_folder) if x.startswith(prefix.replace('validate', 'test'))]
                    _, test_scores =  np.load(os.path.join(model_folder, testscorefile[0]))
                    test_score_from_file = np.average(test_scores)
                    valid_score_from_file = valid_score
        

                        
        if valid_score_from_file > modeldict['max_valid_score']:
            modeldict['max_valid_score'] = valid_score_from_file
            modeldict['scores'] = test_score_from_file
        #if model_args['dataset'].startswith('REDDIT'):
        #    print(modeldict['max_valid_score'], test_score_from_file)
        rows.append(modeldict)
        
    except Exception as e:
        print(f'{e} is wrong w/ {model_folder}')
        errored.append(model_folder)

In [126]:
# load in baselines
if task != 'max_cut':
    baseline_file = 'vc_baseline_scores.csv'
else:
    baseline_file = 'mc_baseline_scores.csv'
baselines = pd.read_csv(baseline_file)

gen_n_dict = dict(zip([50, 100, 400], [[50, 100], [100, 200],[400, 500]]))

# unwind them 
for i, baseline in baselines.iterrows():
    #print(baseline.index)
    for col in baselines.keys():
        if col == 'Unnamed: 0':
            continue
        ds = col
        gen_n = np.nan
        if len(col.split('@@')) > 1:
            ds, gen_n = col.split('@@')
            gen_n = gen_n_dict[int(gen_n)]

        row = {'dataset': ds, 'gen_n': gen_n, 'model_type': baseline['Unnamed: 0'], 'scores': baseline[col], 'baseline': True}
        #print(row)
        rows.append(row)


len(errored)

In [127]:
df = pd.DataFrame.from_records(rows)

from collections import Counter
Counter(df[df.gen_n == 400].model_type)

for x in df.gen_n:
    if not isinstance(x, list) and x !=100:
        print(x)

        

In [128]:
df['gen_n_list'] = df.gen_n
df['gen_n_max'] = df.gen_n_list.apply(lambda x: int(x[1]) if isinstance(x,list) else x)
df.gen_n = df.gen_n.apply(lambda x: int(x[0]) if isinstance(x,list) else x)


from collections import Counter

Counter(df.gen_n_max)

for x, y in zip(df.gen_n, df.gen_n_max):
    if x!=y : print("hi")

In [129]:
df.keys()
# 'problem_type', 'seed',  'prefix', 'RB_n', 'RB_k', 'log_dir',
relevant_keys = [ 'model_type', 'num_layers',
       'repeat_lift_layers', 'num_layers_project', 'rank', 'vc_penalty', 'gen_n', 'gen_n_max',
       'dataset', 'infinite',  'positional_encoding', 'pe_dimension',
       'max_valid_score', 'max_valid_epoch',
       'scores']

df

import matplotlib.pyplot as plt
import seaborn as sns
sns.histplot(df[df.infinite==False].max_valid_epoch)
plt.title('the step (x1000) at which the lowest validation score is achieved (non-infinite)')
plt.xlabel("step (x1000)")
plt.ylabel('count')
plt.show()
df[(df.max_valid_epoch > 90) & (df.infinite==False)][relevant_keys] #.to_csv('230925_checkup_late_bloomers.csv')

sns.histplot(df[df.infinite!=False].max_valid_epoch, bins=range(20))
plt.title('the step (x1000) at which the lowest validation score is achieved (infinite)')
plt.xlabel("step (x1000)")
plt.ylabel('count')
plt.show()
df[(df.max_valid_epoch > 18) & (df.infinite!=False)][relevant_keys] #.to_csv('230925_checkup_late_bloomers.csv')

df.loc[df[df.baseline == False].groupby(['dataset', 'gen_n'])['scores'].idxmax()][relevant_keys]

In [130]:
df.gen_n = df.gen_n.fillna("")
df.gen_n_max = df.gen_n_max.fillna("")

df.dataset

In [131]:
# test models

models_for_test = []

In [132]:
dataset_by_arc = pd.DataFrame()
dss = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz']
mts = ['SDP proj', 'gurobi_2.0', 'gurobi_4.0', 'gurobi_8.0', 'vertex count']
#baseline_names = ['gurobi_2.0', 'gurobi_4.0', 'gurobi_8.0']



for (mt, ds, gen_n), group in df[(df.infinite == False) | (df.dataset == 'ErdosRenyi')].groupby(['model_type', 'dataset', 'gen_n']):
    if mt in mts:
        continue
    if all(group['max_valid_score'].isna()):
        continue
    if ds not in dss:
        gen_n = ""

    #print(mt, ds, gen_n)
    bestidx = group["max_valid_score"].idxmax()
    best_time = df.loc[bestidx]["time_per_pred"]*1000
    mult = 1
    if task == 'vertex_cover':
        mult = -1
    best_score = mult*df.loc[bestidx]['scores']
    score_time_string = f'{best_score:0.2f} ({best_time:0.0f})'
    if ds in dss:
        dataset_by_arc.at[f'{ds}, {gen_n}', mt] = score_time_string
        dataset_by_arc.at[f'{ds}, {gen_n}', 'gen_n'] = gen_n
        dataset_by_arc.at[f'{ds}, {gen_n}', 'dataset'] = ds
    else: 
        dataset_by_arc.at[f'{ds}', mt] = score_time_string
        dataset_by_arc.at[f'{ds}', 'gen_n'] = gen_n
        dataset_by_arc.at[f'{ds}', 'dataset'] = ds

    try:
        models_for_test.append((df.loc[bestidx]['log_dir'], df.loc[bestidx]['dataset'], df.loc[bestidx]['gen_n'])) 
    except Exception as e:
        print(f'{e}')

#dataset_by_arc['dataset'] = dataset_by_arc.index
reorder(dataset_by_arc[[k for k in dataset_by_arc.keys() if k not in ['SDP lift', 'edge count', 'vertex_count']]], by='dataset', columns = ['dataset', 'gen_n', 'GAT', 
                                                                                                                            'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']).to_csv(f'Table1_{task}.csv', index=False) #.style.highlight_max(color = 'green', axis = 1)

In [133]:
dataset_by_arc = pd.DataFrame()
mts = ['GAT', 'GCNN', 'GIN', 'GatedGCNN', 'SDP proj']
dss = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz']

for (mt, ds, gen_n, gen_nmax), group in df[~df.model_type.isin(mts)].groupby(['model_type', 'dataset', 'gen_n', 'gen_n_max']):

    #if mt =='gurobi_4.0': print(ds) # and ds =='MUTAG': print("hi")
    #print(mt, ds, gen_n)
    if all(group['max_valid_score'].isna()) and mt in ['LiftMP'] + mts:
        continue
    if ds not in dss:
        gen_n = ""
        gen_nmax = ""
    #print(mt, ds, gen_n)
    if ds in dss:
        k = f'{ds}_{gen_n}'
    else:
        k = ds

    mult = 1
    if task == 'vertex_cover':
        mult = -1

    if mt not in  ['LiftMP'] + mts:
        assert(len(group) == 1)
        dataset_by_arc.at[k, mt] = mult*group['scores'].max()
    else:
        dataset_by_arc.at[k, 'Type'] = ds
        dataset_by_arc.at[k, 'Nmin'] = gen_n
        dataset_by_arc.at[k, 'Nmax'] = gen_nmax

        bestidx = group["max_valid_score"].idxmax()
        best_time = df.loc[bestidx]["time_per_pred"]*1000
        best_score = mult*df.loc[bestidx]['scores']
        score_time_string = f'{best_score:0.2f} ({best_time:0.0f})'
        dataset_by_arc.at[k, mt] = score_time_string

    try:
        models_for_test.append((df.loc[bestidx]['log_dir'], df.loc[bestidx]['dataset'], df.loc[bestidx]['gen_n'])) 
    except Exception as e:
        print(f'{e}')

#dataset_by_arc.rename(columns={'Nikos': 'CustomLiftProject'}, inplace=True)
reorder(dataset_by_arc[[k for k in dataset_by_arc.keys() if k not in ['SDP lift', 'vertex count', 'edge count']]], by='Type', secondary='Nmin').round(2).to_csv(f'Table2_{task}.csv') #.style.highlight_max(color = 'green', axis = 1)

/tmp/ipykernel_1819292/396912827.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dataset_name_order'] = df[by].map({name: i for i, name in enumerate(canonical_order)})


reorder(dataset_by_arc[[k for k in dataset_by_arc.keys() if k not in ['SDP lift', 'vertex count']]], by='Type', secondary='Nmin').round(2).keys()

models_for_test

df[(df.model_type == 'gurobi_4.0') & (df.dataset == 'ENZYMES')]

for x in models_for_test:
    print(x)